### PART A

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm

from statsmodels.sandbox.regression.gmm import IV2SLS 
from statsmodels.sandbox.regression.gmm import GMM

In [ ]:
input_table = pd.read_csv('https://raw.githubusercontent.com/supragyabajpai/SchulichJourney_DataScience/main/Datasets/midterm_partone.csv')
input_table.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [ ]:
model_iv = sm.OLS(input_table["Inventory Turnover"],input_table[["Constant","Current Ratio","Quick Ratio",\
                                                                 "Debt Asset Ratio"]]).fit()
endog_predict = model_iv.predict(input_table[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]])
input_table["Endogenous Param"] = endog_predict

In [ ]:
model_2sls = sm.OLS(input_table["Stock Change"], input_table[["Constant","Endogenous Param",\
                                                              "Operating Profit","Interaction Effect",\
                                                             ]]).fit()
model_2sls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Stock Change   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     8.530
Date:                Sun, 12 Nov 2023   Prob (F-statistic):           1.27e-05
Time:                        18:23:40   Log-Likelihood:                -1186.5
No. Observations:                1696   AIC:                             2381.
Df Residuals:                    1692   BIC:                             2403.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Constant              -0.0176      0.020     -0.896      0.370      -0.056       0.021
Endogenous Param       0.0011      0.001      1.827      0.068   -7.76e-05       0.002
Operating Profit      -0.1201      0.028     -4.319      0.000      -0.175      -0.066
Interaction Effect     0.0014      0.000      3.621      0.000       0.001       0.002
==============================================================================
Omnibus:                      368.832   Durbin-Watson:                   2.243
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3433.920
Skew:                           0.742   Prob(JB):                         0.00
Kurtosis:                       9.811   Cond. No.                         109.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] 
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()


Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        18:23:41                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

In [ ]:
class gmm_with_delta(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument 

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] 
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] ) * inst[:,0]- delta
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] - delta
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

y_vals = np.array(input_table["Stock Change"])
x_vals = np.array(input_table[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])


beta0_with_delta = np.array([0.1, 0.1, 0.1, 0.1, 0.1]) 
results = gmm_with_delta(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5).fit(beta0_with_delta)
results.summary()

Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 10
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            gmm_with_delta Results                            
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.5862
Model:                 gmm_with_delta   Prob (Hansen J):                 0.444
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        18:25:19                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0208      0.021     -0.986      0.324      -0.062       0.020
p 1            0.0011      0.001      1.839      0.066   -7.31e-05       0.002
p 2           -0.1062      0.032     -3.316      0.001      -0.169      -0.043
p 3            0.0011      0.000      2.688      0.007       0.000       0.002
p 4           -0.0006      0.003     -0.213      0.831      -0.006       0.005
==============================================================================
"""

### PART B

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,recall_score, precision_score, f1_score
import warnings

In [ ]:
def load_and_preprocess_data():
    
    df = pd.read_csv('https://raw.githubusercontent.com/supragyabajpai/SchulichJourney_DataScience/main/Datasets/midterm_parttwo.csv')

    print(df.head(10))

    df = pd.get_dummies(df, columns=['Requested Credit Amount', 'Marital Status', 'Number of Dependents',
                                      'Monthly Income', 'Monthly Expense'], drop_first=True, dtype=int)

    return df

In [ ]:
def split_data(df):

    y = df['Credit Rating']
    X = df.drop('Credit Rating', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

    return X_train, X_test, y_train, y_test

In [ ]:
def train_logistic_regression(X_train, y_train):
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
    grid_search = GridSearchCV(LogisticRegression(random_state=0, max_iter=1000), param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print("Best Parameters:", grid_search.best_params_)
    return best_model

In [ ]:
def evaluate_model(model, X_test, y_test):
    
    y_pred = model.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred, pos_label='Positive')
    precision = precision_score(y_test, y_pred, pos_label='Positive')
    f1 = f1_score(y_test, y_pred, pos_label='Positive')

    print("Confusion Matrix:\n", conf_matrix)
    print("\nRecall:", recall)
    print("Precision:", precision)
    print("F1 Score:", f1)

In [ ]:
def adjust_threshold_and_evaluate(model, X_test, y_test, desired_approval_rate=0.15):

    y_probs = model.predict_proba(X_test)[:, 1]
    threshold = np.percentile(y_probs, 100 - (desired_approval_rate * 100))

    y_pred_updated = (y_probs >= threshold).astype(int)

    y_test = pd.get_dummies(y_test, dtype=int, drop_first=True)

    conf_matrix = confusion_matrix(y_test, y_pred_updated)
    recall = recall_score(y_test, y_pred_updated)
    precision = precision_score(y_test, y_pred_updated)
    f1 = f1_score(y_test, y_pred_updated)

    print("Confusion Matrix:\n", conf_matrix)
    print("\nRecall:", recall)
    print("Precision:", precision)
    print("F1 Score:", f1)

Loading the data

In [ ]:
df = load_and_preprocess_data()


   Years of Education after High School Requested Credit Amount   
0                                     1                     Low  \
1                                     2                     Low   
2                                     1                     Low   
3                                     3                     Low   
4                                     3                     Low   
5                                     2                     Low   
6                                     2                     Low   
7                                     1                     Low   
8                                     3                     Low   
9                                     1                     Low   

  Number of Dependents Monthly Income Monthly Expense Marital Status   
0         No dependent       Very low        Very low        Married  \
1         No dependent       Very low        Very low         Single   
2         No dependent       Very low        V

Splitting the data in training and testing

In [ ]:
X_train, X_test, y_train, y_test = split_data(df)


Trainging the base model, before setting the threshold

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    
    model = train_logistic_regression(X_train, y_train)

Best Parameters: {'C': 0.001}


Calculating the confusion matrix before setting the threshold.


In [ ]:
evaluate_model(model, X_test, y_test)

Confusion Matrix:
 [[   0  577]
 [   0 3464]]

Recall: 1.0
Precision: 0.8572135609997525
F1 Score: 0.9231179213857428


Calculating the confusion matrix after setting the threshold.

In [ ]:
adjust_threshold_and_evaluate(model, X_test, y_test)


Confusion Matrix:
 [[ 506   71]
 [2925  539]]

Recall: 0.15560046189376445
Precision: 0.8836065573770492
F1 Score: 0.26460481099656363


The credit approval model's performance, as indicated by the provided results, demonstrates a trade-off between precision and recall. With a recall of 0.1556, the model captures only about 15.56% of all actual positive cases (approved applications), suggesting a limitation in its ability to identify true positives. On the other hand, the model exhibits high precision (0.8836), indicating that when it predicts an approval, it is correct approximately 88.36% of the time. However, this high precision comes at the expense of overall recall, resulting in an F1 Score of 0.2646. This score reflects a balance between precision and recall, emphasizing the need for further optimization to enhance the model's ability to identify a greater proportion of approved applications without compromising precision.